In [1]:
import pandas as pd
from tqdm.auto import tqdm
from transformers import pipeline

2023-11-26 02:41:09.008945: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-26 02:41:09.398106: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-26 02:41:10.353356: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
with open("../data/metadata.py") as fp:
    data = fp.read()
    METADATA = eval(data.replace("\n", ""))

In [3]:
model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
model_name = "yiyanghkust/finbert-tone"

nlp = pipeline("sentiment-analysis", model=model_name)

MAP = {
    "Positive": 1,
    "Negative": -1,
    "neutral": 0
}

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [4]:
def sentiment_article(article):
    pieces = article.split("\n")
    analysis = nlp(pieces)
    analysis = pd.DataFrame(analysis)
    analysis["factor"] = analysis["label"].map(MAP)
    analysis["new_score"] = analysis["score"]*analysis["factor"]
    return analysis["new_score"].mean()

In [6]:
for data in METADATA:

    LINK = data["link"]
    TICKER = data["ticker"]
    print(LINK, TICKER)

    data = pd.read_csv(f"../data/raw_data/raw_data_{TICKER}.csv")
    data["Date"] = pd.to_datetime(data["Date"])
    data["len"] =  data["text"].apply(lambda x: len(x.split()))
    
    sentiment = []
    for _, row in tqdm(data.iterrows(), total=len(data)):
        try:
            sentiment.append(sentiment_article(row["text"]))
        except:
            sentiment.append(0)

    data["sentiment"] = sentiment

    data.to_csv(f"../data/final_data/final_data_{TICKER}.csv", index=False)

https://www.moneycontrol.com/news/tags/hdfc.html/news/ HDFC


/tmp/ipykernel_30443/1495049287.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["Date"] = pd.to_datetime(data["Date"])
/tmp/ipykernel_30443/1495049287.py:8: FutureWarning: Parsing '{res.tzname}' as tzlocal (dependent on system timezone) is deprecated and will raise in a future version. Pass the 'tz' keyword or call tz_localize after construction instead
  data["Date"] = pd.to_datetime(data["Date"])


  0%|          | 0/25 [00:00<?, ?it/s]